# T3.2 Analizar e visualizar estatísticas de datos baixados de MongoDB
## Ralphy Núñez Mercado
## Conexión a MongoDB dende Python e análise de datos

En Atlas mete o dataset de exemplo na túa base de datos.

Engade o enderezo IP autorizado a conectar para que poidas conectar á base de datos.

Proba a conectar dende compass coa URL de conexión que che da.

Da BBDD de proba: sample_mfix, na colección movies: https://www.mongodb.com/docs/atlas/sample-data/sample-mflix/

Conecta a Atlas dende Python, carga a colección en Pandas. Averigua o modo máis adecuado.

In [29]:
# %conda install -y pymongo dnspython
from pymongo import MongoClient
import json
import pandas as pd

#HOST="(...).mongodb.net"
HOST="cluster0.6ej71.mongodb.net"
PORT=27017
USERNAME="ralphynm1000"
PASSWORD="hola1234"

if HOST == 'localhost':
    if not USERNAME:
        cli_mongo = MongoClient(HOST, PORT)
    else:
        cli_mongo = MongoClient(HOST, PORT, USERNAME, PASSWORD)
else:
    cli_mongo = MongoClient(f"mongodb+srv://{USERNAME}:{PASSWORD}@{HOST}/")

db_sample_mflix = cli_mongo['sample_mflix']
movies = db_sample_mflix['movies']


A) Contar o total de películas.


In [30]:
movies.estimated_document_count({})

21349

B) Contar o número de películas de cada xénero.

In [31]:
generos = movies.distinct('genres')

data = {}
for i in generos:
    data[i] = movies.count_documents({"genres": i})

# convertir diccionario en dataFrame
df = pd.DataFrame(list(data.items()), columns=['Género', 'Cantidad'])

df

,Género,Cantidad
0,Action,2381
1,Adventure,1900
2,Animation,912
3,Biography,1269
4,Comedy,6532
5,Crime,2457
6,Documentary,1834
7,Drama,12385
8,Family,1249
9,Fantasy,1055


C) Contar cantas películas hai por ano.

In [32]:
documentos = list(movies.find())

df = pd.DataFrame(documentos)

df["year"] = df["released"].dt.year

contar_pelis_año = df["year"].value_counts()

contar_pelis_año


year
2014.0    1089
2013.0    1063
2011.0     909
2012.0     903
2008.0     871
          ... 
1913.0       1
1909.0       1
1918.0       1
1919.0       1
1903.0       1
Name: count, Length: 107, dtype: int64

D) Mirar se hai correlación entre imdb e rotten tomatoes.

In [33]:
df = pd.DataFrame(documentos)

# Extraer la calificación de IMDb (rating) del campo 'imdb'.
df['imdb_rating'] = df['imdb'].apply(
    lambda x: x.get('rating') if isinstance(x, dict) else None
)

# Extraer la calificación de la crítica (critic rating) de Rotten Tomatoes.
df['rt_critic_rating'] = df['tomatoes'].apply(
    lambda x: x.get('critic', {}).get('rating') if isinstance(x, dict) and 'critic' in x else None
)

# Eliminar filas con valores faltantes
df_clean = df.dropna(subset=['imdb_rating', 'rt_critic_rating'])

# Convertir a valores numéricos
df_clean.loc[:, 'imdb_rating'] = pd.to_numeric(df_clean['imdb_rating'], errors='coerce')
df_clean.loc[:, 'rt_critic_rating'] = pd.to_numeric(df_clean['rt_critic_rating'], errors='coerce')


# Eliminar NaNs
df_clean = df_clean.dropna(subset=['imdb_rating', 'rt_critic_rating'])

correlacion = df_clean['imdb_rating'].corr(df_clean['rt_critic_rating'])

print(f"Correlación IMDb - Rotten Tomatoes: {correlacion:.2f}")



Correlación IMDb - Rotten Tomatoes: 0.75


E) Contar cantas películas ten cada director

In [40]:
df = pd.DataFrame(documentos)

# Asegurarse de que 'directors' no sea nulo
df = df[df['directors'].notna()]

# Expandir la lista de directores a filas individuales
df_exploded = df.explode('directors').reset_index(drop=True)

# Contar cuántas películas tiene cada director
contar_pelis_directores = df_exploded['directors'].value_counts()

contar_pelis_directores



directors
Woody Allen         40
Martin Scorsese     32
Takashi Miike       31
Sidney Lumet        29
Steven Spielberg    29
                    ..
Cècile Telerman      1
Jeeva                1
Larry Bishop         1
Diane Bertrand       1
Marcin Koszalka      1
Name: count, Length: 10843, dtype: int64

F) Contar en cantas películas ten participado cada actor

In [35]:
df = pd.DataFrame(documentos)

# Filtrar filas donde 'cast' no sea nulo
df = df[df['cast'].notna()]

# Expandir la lista de actores a filas individuales
df_exploded = df.explode('cast').reset_index(drop=True)

# Contar cuántas películas tiene cada actor
contar_pelis_actores = df_exploded['cast'].value_counts()

print(contar_pelis_actores)

cast
Gèrard Depardieu        67
Robert De Niro          58
Michael Caine           51
Bruce Willis            49
Samuel L. Jackson       48
                        ..
Margrèt èkadèttir        1
Sibylle Canonica         1
Stefanie von Pfetten     1
Stellina Rusich          1
Chantal Akerman          1
Name: count, Length: 38636, dtype: int64


G) Contar cantas películas hai de cada idioma

In [36]:
df = pd.DataFrame(documentos)

# Filtrar filas donde 'languages' no sea nulo
df = df[df['languages'].notna()]

# Expandir la lista de idiomas a filas individuales
df_exploded = df.explode('languages').reset_index(drop=True)

# Contar cuántas películas hay en cada idioma
pelis_por_idioma = df_exploded['languages'].value_counts()

print(pelis_por_idioma)

languages
English      14639
French        2405
Spanish       1600
German        1377
Italian       1228
             ...  
Quenya           1
Uighur           1
Visayan          1
Kabyle           1
Purepecha        1
Name: count, Length: 231, dtype: int64


H) Cal é a media de premios das películas

In [37]:
df = pd.DataFrame(documentos)

# Extraer só as películas que teñen 'awards.wins'
df = df[df['awards'].notna()]
df = df[df['awards'].apply(lambda x: 'wins' in x and isinstance(x['wins'], (int, float)))]

# Calcular a media
media_premios = df['awards'].apply(lambda x: x['wins']).mean()

print(f"Media de premios (vitorias) por película: {media_premios:.2f}")

Media de premios (vitorias) por película: 4.06


J) Amosar as películas que teñen alomenos 3 premios

In [38]:
df = pd.DataFrame(documentos)

# Filtrar películas que teñen o campo 'awards.wins' definido como número
df = df[df['awards'].notna()]
df = df[df['awards'].apply(lambda x: 'wins' in x and isinstance(x['wins'], (int, float)))]

# Quedar só coas que teñen 3 ou máis premios
pelis_con_3_premios = df[df['awards'].apply(lambda x: x['wins'] >= 3)]

# Amosar título e número de premios
pelis_con_3_premios[['title', 'awards']]

,title,awards
44,Napoleon,"{'wins': 3, 'nominations': 0, 'text': '3 wins.'}"
46,Wings,"{'wins': 5, 'nominations': 1, 'text': 'Won 2 O..."
68,Morocco,"{'wins': 7, 'nominations': 0, 'text': 'Nominat..."
69,Romance,"{'wins': 3, 'nominations': 0, 'text': 'Nominat..."
74,Cimarron,"{'wins': 5, 'nominations': 4, 'text': 'Won 3 O..."
...,...,...
21304,Tikkun,"{'wins': 7, 'nominations': 1, 'text': '7 wins ..."
21317,The High Sun,"{'wins': 8, 'nominations': 3, 'text': '8 wins ..."
21330,Bitcoin: The End of Money as We Know It,"{'wins': 3, 'nominations': 0, 'text': '3 wins.'}"
21331,Land and Shade,"{'wins': 4, 'nominations': 3, 'text': '4 wins ..."


K) Contar as películas por país

In [39]:
pais = movies.distinct('countries')

data = {}
for i in pais:
    data[i] = movies.count_documents({"countries": i})

df = pd.DataFrame(list(data.items()), columns=['País', 'Peliculas'])

df

,País,Peliculas
0,Afghanistan,8
1,Albania,3
2,Algeria,13
3,Angola,3
4,Argentina,195
...,...,...
153,Vietnam,9
154,West Germany,246
155,Yugoslavia,47
156,Zaire,1
